# SenSwiss 2021

Code to run in parallel with the acquisition of images in SpinView.

Instructions: 

0. Check that the exposure time of the camera is constant
1. Check you have a folder named focus at the main path
2. Create a new folder for the measurement (this needs to be done each time, not in advance!!! (code checks last created folder))
3. Save image for the focus in SpinView in the folder ./focus as type .tiff
4. Run run_parallel.py
5. Select ROIs and press the cross
6. Start the acquisition in SpinView, saving the images in the folder you created in step 2 as type .tiff
7. Look that everything is okay in the terminal
8. Press s (for like 1-2seconds) when you want to finish computing the preprocessing and analysis and save the results in result.csv, which will be placed in the folder of step 2.

1. Set everything up in SpinView:
    * `/focus`
    * `/images/NEW_FOLDER_FOR_NEW_TEST`
    * exposure auto OFF, 11113us
    
2. Take an image for the focus before flowing BB (just in case)

3. Take another image if you moved the chip

4. Once you are sure the focus is the one you want, press **RUN in run_parallel** notebook

5. After, **start** the acquisition in **SpinView** (do not forget this step!!!)

## Integration with pump
1. connect with port
2. initialise
3. cleaning
4. pull BB
5. push pull sample
6. start recording SpinView
7. run parallel

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import keyboard
import sys
import serial
import time

import run_parallel
from pump import initialise_LSPone, general_cleaning_procedure, pick_BB
from run_parallel import describe_folders, press_ROIs, run_analysis

## Pump

To connect the pump to the computer (only done at the very beginning of times).

In [ ]:
# Open serial connection -> check COM port on your device
lsp = serial.Serial('COM5', 9600, timeout=1000)  # TODO: change the comport depending on your computer
print('LSPone connected on ', lsp.name)

# Initialise LSPone normally already done
initialise_LSPone(lsp)

# 0. General Cleaning # this were i start
# 2: PBS, 3: Air, 4: Ethanol   (order of the tubes in the pump)
general_cleaning_procedure(lsp)

## Starting the measurements

Aspiring the Blocking Buffer

In [ ]:
# Sucking BB at 50 ul/min
pick_BB(lsp)

Flowing the sample

In [ ]:
# Push and pull sample at 50 ul/min
push_pull_sample(lsp, n_times = 4)

## SpinView --> start recording

## Select ROI, preprocess and analysis

In [ ]:
# Select ROI
ORIGINAL_FOLDER, IMG_PROCESSED_FOLDER, IMG_FOLDER, DIR_ROI, DIR = describe_folders()
ROIs = press_ROIs(DIR_ROI, ORIGINAL_FOLDER)

In [ ]:
# Run processing and analysis
run_analysis(ROIs, IMG_FOLDER=IMG_FOLDER, DIR=DIR, window_size=5, framerate=2, threshold=130)

In [ ]:
# Running in parallel
run_parallel.main()

In [ ]:
# Loading results
result_df = pd.read_csv('./images/20210828/result.csv')   # TODO CHANGE NAME
result_df['Signal'].plot()